# 1️⃣ Train CNN on CIFAR-10 (PyTorch)

This notebook trains a lightweight CNN for CIFAR-10 and saves it as `base_model.pt`.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
root = '/content/drive/MyDrive/hardware_aware_optimization'
os.makedirs(f"{root}/notebooks", exist_ok=True)
os.makedirs(f"{root}/models", exist_ok=True)
os.makedirs(f"{root}/data/cifar10", exist_ok=True)
os.makedirs(f"{root}/results", exist_ok=True)
print("Project folders created!")

Mounted at /content/drive
Project folders created!


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import torchvision
import torchvision.transforms as transforms

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [ ]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

In [ ]:
trainset = torchvision.datasets.CIFAR10(root=f'{root}/data/cifar10', train=True, download=True, transform=transform_train)
testset = torchvision.datasets.CIFAR10(root=f'{root}/data/cifar10', train=False, download=True, transform=transform_test)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

100%|██████████| 170M/170M [00:13<00:00, 12.9MB/s]


In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(128*8*8, 256),
            nn.ReLU(),
            nn.Linear(256, 10)
        )
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

model = SimpleCNN().to(device)
print(model)

SimpleCNN(
  (features): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU()
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=8192, out_features=256, bias=True)
    (2): ReLU()
    (3): Linear(in_features=256, out_features=10, bias=True)
  )
)


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = StepLR(optimizer, step_size=20, gamma=0.5)

In [ ]:
def train(model, trainloader, criterion, optimizer, epoch):
    model.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(trainloader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"[Epoch {epoch}] Loss: {running_loss/len(trainloader):.4f}")

def test(model, testloader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    acc = 100 * correct / total
    print(f"Test Accuracy: {acc:.2f}%")
    return acc

In [ ]:
best_acc = 0
for epoch in range(1, 26):
    train(model, trainloader, criterion, optimizer, epoch)
    acc = test(model, testloader)
    scheduler.step()
    if acc > best_acc:
        best_acc = acc
        torch.save(model.state_dict(), f"{root}/models/base_model.pt")
        print("[Saved best model]")
print(f"Best test accuracy: {best_acc:.2f}%")

[Epoch 1] Loss: 1.5970
Test Accuracy: 51.30%
[Saved best model]
[Epoch 2] Loss: 1.1974
Test Accuracy: 61.06%
[Saved best model]
[Epoch 3] Loss: 1.0596
Test Accuracy: 65.84%
[Saved best model]
[Epoch 4] Loss: 0.9835
Test Accuracy: 70.45%
[Saved best model]
[Epoch 5] Loss: 0.9367
Test Accuracy: 71.55%
[Saved best model]
[Epoch 6] Loss: 0.8911
Test Accuracy: 72.62%
[Saved best model]
[Epoch 7] Loss: 0.8570
Test Accuracy: 74.65%
[Saved best model]
[Epoch 8] Loss: 0.8350
Test Accuracy: 75.04%
[Saved best model]
[Epoch 9] Loss: 0.7989
Test Accuracy: 74.66%
[Epoch 10] Loss: 0.7882
Test Accuracy: 74.01%
[Epoch 11] Loss: 0.7685
Test Accuracy: 75.07%
[Saved best model]
[Epoch 12] Loss: 0.7443
Test Accuracy: 75.05%
[Epoch 13] Loss: 0.7315
Test Accuracy: 75.69%
[Saved best model]
[Epoch 14] Loss: 0.7155
Test Accuracy: 76.89%
[Saved best model]
[Epoch 15] Loss: 0.7033
Test Accuracy: 76.54%
[Epoch 16] Loss: 0.6857
Test Accuracy: 78.09%
[Saved best model]
[Epoch 17] Loss: 0.6708
Test Accuracy: 79.54%